In [1]:
# -*- coding: utf-8 -*-
import os
import csv
from music21 import note
import Utils.KeyPos as KP

os.makedirs(f"./Data./fingerings_csv", exist_ok = True)
files = os.listdir("./Statistical_Learning_and_Estimation_of_Piano_Fingering./PianoFingeringDataset_v1.2/FingeringFiles")
pieces = []
fieldnames = [
    "Note_number",
    "Onset_time_in_sec",
    "Offset_time_in_sec",
    "Pitch",
    "Pitch_name",
    "KeyPos_x",
    "KeyPos_y",
    "Onset_velocity",
    "Offset_velocity",
    "Begin_fingering",
    "End_fingering"
]

for file in files:
    with open(f"./Statistical_Learning_and_Estimation_of_Piano_Fingering./PianoFingeringDataset_v1.2./FingeringFiles./{file}", "r") as txt_file:
        line = txt_file.readline() # read the first useless line
        # write csv
        with open(f"./Data./fingerings_csv./{file[:-4]}.csv", "w", newline = "") as csv_file:
            csv_writer = csv.DictWriter(csv_file, fieldnames = fieldnames)
            csv_writer.writeheader()
            for line in txt_file:
                temp = line.split()
                # read each feature
                features = {
                    "Note_number": float(temp[0]),
                    "Onset_time_in_sec": float(temp[1]),
                    "Offset_time_in_sec": float(temp[2]),
                    "Pitch": float(note.Note(temp[3]).pitch.midi),
                    "Pitch_name": temp[3], # just easier to read
                    "KeyPos_x": None,
                    "KeyPos_y": None,
                    "Onset_velocity": float(temp[4]),
                    "Offset_velocity": float(temp[5]),
                    "Begin_fingering": temp[7],
                    "End_fingering": None
                }

                # set Key position
                keypos = KP.PitchToKeyPos(features["Pitch"])
                features["KeyPos_x"] = float(keypos.x)
                features["KeyPos_y"] = float(keypos.y)

                # if same note with different fingering
                if "_" in features["Begin_fingering"]:
                    fingerings = features["Begin_fingering"].split("_")
                    if fingerings[0] == "":
                        features["Begin_fingering"] = float(fingerings[1])
                        features["End_fingering"] = features["Begin_fingering"]
                    elif fingerings[1] == "":
                        features["Begin_fingering"] = float(fingerings[0])
                        features["End_fingering"] = features["Begin_fingering"]
                    else:
                        features["Begin_fingering"] = float(fingerings[0])
                        features["End_fingering"] = float(fingerings[1])
                else:
                    features["Begin_fingering"] = float(features["Begin_fingering"])
                    features["End_fingering"] = features["Begin_fingering"]
                csv_writer.writerow(features)